In [1]:
# importing relevant packages

import pandas as pd
import numpy as np
import re

import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.feature_extraction.text import CountVectorizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



In [2]:
# reading data
df_2022 = pd.read_csv('../data/2022_twitter_df.csv')
df_2018 = pd.read_csv('../data/2018_twitter_df.csv')
df_2020 = pd.read_csv('../data/2020_twitter_df.csv')



In [3]:
#restricting text from tweets to alpha
df_2022["Embedded_text"] = df_2022['Embedded_text'].str.replace('[^\\w\\s]','', regex=True)
df_2022["Embedded_text"] = df_2022['Embedded_text'].str.replace('\\d+','', regex=True)

## your code here to define function
def sent_scores(string_ex):
    doc = nlp(string_ex)

    # getting start and end characters for the named entities
    ents_as_spans = [(ent.start_char, ent.end_char) for ent in doc.ents]

    new_list = []
    i = 0
    for start, end in sorted(ents_as_spans):
        #add all chars before this entity
        if i < start:
            new_list.append(string_ex[i:start])
        i = end
    if i < len(string_ex):
        new_list.append(string_ex[i:])
    
    removed_ents = "".join(new_list)

    
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_ex = sent_obj.polarity_scores(removed_ents)
    return sentiment_ex

sent = [sent_scores(text) for text in df_2022["Embedded_text"]]


In [4]:
df_2022["sent_scores"] = sent
df_2022
df_2022["compound_scores"] = [x['compound'] for x in df_2022["sent_scores"]]
df_2022


,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL,sent_scores,compound_scores
0,Lauren Boebert,@laurenboebert,2022-01-17T23:32:38.000Z,Lauren Boebert\n@laurenboebert\n·\nJan 18,The only solution Ive ever heard the Left prop...,NaN,"1,683","2,259",11.7K,[],https://twitter.com/laurenboebert/status/14832...,"{'neg': 0.078, 'neu': 0.75, 'pos': 0.172, 'com...",0.5279
1,Catherine,@catherine___c,2022-01-17T22:54:02.000Z,Catherine\n@catherine___c\n·\nJan 17,Climate change doesnt cause volcanic eruptions...,NaN,158,64,762,[],https://twitter.com/catherine___c/status/14832...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,king Keith,@KaConfessor,2022-01-17T23:51:41.000Z,king Keith\n@KaConfessor\n·\nJan 18,Vaccinated tennis ball boy collapses in the te...,NaN,24,118,159,['https://pbs.twimg.com/ext_tw_video_thumb/148...,https://twitter.com/KaConfessor/status/1483225...,"{'neg': 0.155, 'neu': 0.845, 'pos': 0.0, 'comp...",-0.2960
3,PETRIFIED CLIMATE PARENT,@climate_parent,2022-01-17T21:42:04.000Z,PETRIFIED CLIMATE PARENT\n@climate_parent\n·\n...,North America has experienced an average winte...,NaN,15,50,158,[],https://twitter.com/climate_parent/status/1483...,"{'neg': 0.0, 'neu': 0.91, 'pos': 0.09, 'compou...",0.2924
4,Thomas Speight,@Thomas_Sp8,2022-01-17T21:10:40.000Z,Thomas Speight\n@Thomas_Sp8\n·\nJan 17,Theyre gonna do the same with Climate Change w...,🅾,4,24,127,['https://pbs.twimg.com/profile_images/1544171...,https://twitter.com/Thomas_Sp8/status/14831850...,"{'neg': 0.117, 'neu': 0.834, 'pos': 0.049, 'co...",-0.4939
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9045,Dr Srijana Mitra Das,@srijanapiya17,2022-07-18T12:08:28.000Z,Dr Srijana Mitra Das\n@srijanapiya17\n·\nJul 18,ClimateChange is now the greatest story on Ear...,NaN,2,16,24,['https://pbs.twimg.com/profile_images/5140754...,https://twitter.com/srijanapiya17/status/15490...,"{'neg': 0.0, 'neu': 0.92, 'pos': 0.08, 'compou...",0.6369
9046,1%_Better_Every_Day,@jh336405,2022-07-18T00:33:20.000Z,1%_Better_Every_Day\n@jh336405\n·\nJul 18,Replying to \njh\n acuna_r\n and others\nAnd ...,💯 💯 🌏,4,NaN,NaN,['https://pbs.twimg.com/profile_images/1442412...,https://twitter.com/jh336405/status/1548828230...,"{'neg': 0.0, 'neu': 0.911, 'pos': 0.089, 'comp...",0.4391
9047,David Schechter,@DavidSchechter,2022-07-18T21:13:13.000Z,David Schechter\n@DavidSchechter\n·\nJul 18,While Texans are being asked to use less elect...,NaN,3,14,23,['https://pbs.twimg.com/card_img/1549138950475...,https://twitter.com/DavidSchechter/status/1549...,"{'neg': 0.12, 'neu': 0.839, 'pos': 0.041, 'com...",-0.5106
9048,Daily Climate,@TheDailyClimate,2022-07-18T10:15:09.000Z,Daily Climate\n@TheDailyClimate\n·\nJul 18,Sea levels are rising and communities are scra...,NaN,NaN,3,NaN,['https://pbs.twimg.com/card_img/1547862999808...,https://twitter.com/TheDailyClimate/status/154...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


In [5]:
#restricting text from tweets to alpha
df_2018["Embedded_text"] = df_2018['message'].str.replace('[^\\w\\s]','', regex=True)
df_2018["Embedded_text"] = df_2018['Embedded_text'].str.replace('\\d+','', regex=True)
df_2018["Embedded_text"] = df_2018["Embedded_text"].str.lower()
df_2018

,sentiment,message,tweetid,Embedded_text
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840,tiniebeany climate change is an interesting hu...
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641,rt natgeochannel watch beforetheflood right he...
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256,fabulous leonardo dicaprios film on climate ch...
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904,rt mick_fanning just watched this amazing docu...
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153,rt cnalive pranita biswasi a lutheran from odi...
...,...,...,...,...
43938,1,"Dear @realDonaldTrump,\nYeah right. Human Medi...",791307031919550464,dear realdonaldtrump\nyeah right human mediate...
43939,1,What will your respective parties do to preven...,791316857403936768,what will your respective parties do to preven...
43940,2,RT @MikkiL: UN Poll Shows Climate Change Is th...,791357509101621249,rt mikkil un poll shows climate change is the ...
43941,0,RT @taehbeingextra: i still can$q$t believe th...,791390042136641537,rt taehbeingextra i still canqt believe this g...


In [ ]:
## your code here to define function
def sent_scores(string_ex):
    doc = nlp(string_ex)

    # getting start and end characters for the named entities
    ents_as_spans = [(ent.start_char, ent.end_char) for ent in doc.ents]

    new_list = []
    i = 0
    for start, end in sorted(ents_as_spans):
        #add all chars before this entity
        if i < start:
            new_list.append(string_ex[i:start])
        i = end
    if i < len(string_ex):
        new_list.append(string_ex[i:])
    
    removed_ents = "".join(new_list)

    
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_ex = sent_obj.polarity_scores(removed_ents)
    return sentiment_ex

sent = [sent_scores(text) for text in df_2018["Embedded_text"]]

In [14]:

df_2018["sent_scores"]
df_2018
df_2018["compound_scores"] = [x['compound'] for x in df_2018["sent_scores"]]
df_2018


,sentiment,message,tweetid,Embedded_text,sent_scores,compound_scores
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840,tiniebeany climate change is an interesting hu...,"{'neg': 0.081, 'neu': 0.622, 'pos': 0.297, 'co...",0.6428
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641,rt natgeochannel watch beforetheflood right he...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256,fabulous leonardo dicaprios film on climate ch...,"{'neg': 0.0, 'neu': 0.703, 'pos': 0.297, 'comp...",0.5859
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904,rt mick_fanning just watched this amazing docu...,"{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compou...",0.6705
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153,rt cnalive pranita biswasi a lutheran from odi...,"{'neg': 0.149, 'neu': 0.747, 'pos': 0.104, 'co...",-0.2732
...,...,...,...,...,...,...
43938,1,"Dear @realDonaldTrump,\nYeah right. Human Medi...",791307031919550464,dear realdonaldtrump\nyeah right human mediate...,"{'neg': 0.0, 'neu': 0.696, 'pos': 0.304, 'comp...",0.5859
43939,1,What will your respective parties do to preven...,791316857403936768,what will your respective parties do to preven...,"{'neg': 0.0, 'neu': 0.577, 'pos': 0.423, 'comp...",0.6808
43940,2,RT @MikkiL: UN Poll Shows Climate Change Is th...,791357509101621249,rt mikkil un poll shows climate change is the ...,"{'neg': 0.191, 'neu': 0.809, 'pos': 0.0, 'comp...",-0.3818
43941,0,RT @taehbeingextra: i still can$q$t believe th...,791390042136641537,rt taehbeingextra i still canqt believe this g...,"{'neg': 0.078, 'neu': 0.741, 'pos': 0.181, 'co...",0.3612


In [8]:
#restricting text from tweets to alpha
df_2020["Embedded_text"] = df_2020['text'].str.replace('[^\\w\\s]','', regex=True)
df_2020["Embedded_text"] = df_2020['Embedded_text'].str.replace('\\d+','', regex=True)

## your code here to define function
def sent_scores(string_ex):
    doc = nlp(string_ex)

    # getting start and end characters for the named entities
    ents_as_spans = [(ent.start_char, ent.end_char) for ent in doc.ents]

    new_list = []
    i = 0
    for start, end in sorted(ents_as_spans):
        #add all chars before this entity
        if i < start:
            new_list.append(string_ex[i:start])
        i = end
    if i < len(string_ex):
        new_list.append(string_ex[i:])
    
    removed_ents = "".join(new_list)

    
    sent_obj = SentimentIntensityAnalyzer()
    sentiment_ex = sent_obj.polarity_scores(removed_ents)
    return sentiment_ex

sent = [sent_scores(text) for text in df_2020["Embedded_text"]]


In [9]:
df_2020["sent_scores"] = sent
df_2020
df_2020["compound_scores"] = [x['compound'] for x in df_2020["sent_scores"]]
df_2020


,id,date,retweets,source,author,likes,text,twitter_name,location,verified,followers,friends,polarity,subjectivity,Embedded_text,sent_scores,compound_scores
0,2184934963,2020-12-22 23:22:20,71,Twitter Web App,GO GREEN,91,The death of summer Arctic ice our Earth coole...,ECOWARRIORSS,NaN,False,23415,20439,-0.054365,0.426984,The death of summer Arctic ice our Earth coole...,"{'neg': 0.135, 'neu': 0.809, 'pos': 0.056, 'co...",-0.7003
1,508658626,2020-12-10 14:30:00,14,Twitter for Advertisers,Elsevier Energy,98,Elsevier and the EditorsinChief are pleased to...,ElsevierEnergy,"Oxford, England",False,6615,508,0.387500,0.633333,Elsevier and the EditorsinChief are pleased to...,"{'neg': 0.0, 'neu': 0.718, 'pos': 0.282, 'comp...",0.6249
2,2607105006,2020-12-22 21:28:52,0,Twitter Web App,Arwyn Thomas,1,From better climate change education to improv...,siwarr5,Carmarthen,False,22,133,0.261905,0.345238,From better climate change education to improv...,"{'neg': 0.0, 'neu': 0.621, 'pos': 0.379, 'comp...",0.8750
3,19609660,2020-12-22 21:24:10,0,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",0,climate change Links to FIXING CLIMATE CHANGE ...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,climate change Links to FIXING CLIMATE CHANGE ...,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp...",0.1531
4,19609660,2020-12-21 22:52:09,1,Twitter Web App,"Tom Gillispie, EDITOR/WRITER",1,climate change The 11TH HOUR FOR THE EARTH cli...,EDITORatWORK,"Rural Hall, North Carolina, USA",False,4191,3708,0.000000,0.000000,climate change The TH HOUR FOR THE EARTH clima...,"{'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'comp...",0.1531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,54992908,2020-01-03 12:29:11,0,Twitter Web App,Sascha Becher,0,An Inconsistent Truth From global cooling 1970...,SaschaBecher,Deutschland,False,132,132,0.000000,0.000000,An Inconsistent Truth From global cooling s to...,"{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'comp...",0.4404
392,209636315,2020-01-02 08:38:37,0,Twitter for iPhone,Morgan Fitzgibbon,0,2 climate thought alongside Einstein Robert Op...,morganefitz,"Cork, Ireland",False,286,156,0.194444,0.494444,climate thought alongside Einstein Robert Opp...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
393,209636315,2020-01-02 08:35:52,0,Twitter for iPhone,Morgan Fitzgibbon,0,3 climate used as supporting evidence that man...,morganefitz,"Cork, Ireland",False,286,156,0.286667,0.466667,climate used as supporting evidence that man ...,"{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.5859
394,21769361,2020-01-02 03:47:08,0,Twitter for iPhone,STEⓋE BOZIC,0,Global warming Global government via YouTube c...,STEVE_BOZIC,"ÜT: 43.863963,-79.749634",False,478,738,0.000000,0.000000,Global warming Global government via YouTube c...,"{'neg': 0.0, 'neu': 0.686, 'pos': 0.314, 'comp...",0.2960


In [15]:
df_2018.to_csv("../output/2018_sentiment.csv")
df_2022.to_csv("../output/2022_sentiment.csv")
df_2020.to_csv("../output/2020_sentiment.csv")